In [1]:
import os

In [2]:
%pwd

'c:\\Users\\Ayaz Khan\\Desktop\\Chest-Cancer-Classification\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\Ayaz Khan\\Desktop\\Chest-Cancer-Classification'

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    training_data: Path
    params_epochs: int
    params_batch_size: int
    params_is_augmentation: bool
    params_image_size: list
    params_learning_rate: float

In [6]:
import os
import sys

In [7]:
sys.path.append(os.path.abspath("src"))

In [8]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories
import tensorflow as tf

In [9]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

        

    def get_training_config(self) -> TrainingConfig:
        training = self.config.training
        prepare_base_model = self.config.prepare_base_model
        params = self.params
        training_data = os.path.join(self.config.data_ingestion.unzip_dir, "Chest-CT-Scan-data")
        create_directories([
            Path(training.root_dir)
        ])

        training_config = TrainingConfig(
            root_dir=Path(training.root_dir),
            trained_model_path=Path(training.trained_model_path),
            updated_base_model_path=Path(prepare_base_model.updated_base_model_path),
            training_data=Path(training_data),
            params_epochs=params.EPOCHS,
            params_batch_size=params.BATCH_SIZE,
            params_is_augmentation=params.AUGMENTATION,
            params_image_size=params.IMAGE_SIZE
        )

        return training_config

In [10]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time

In [11]:
import tensorflow as tf
from pathlib import Path
from cnnClassifier.entity.config_entity import TrainingConfig

class Training:
    def __init__(self, config: TrainingConfig):
        self.config = config

    def get_base_model(self):
        self.model = tf.keras.models.load_model(
            self.config.updated_base_model_path
        )

    def train_valid_generator(self):

        datagenerator_kwargs = dict(
            rescale=1./255,
            validation_split=0.20
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

        if self.config.params_is_augmentation:
            train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range=40,
                horizontal_flip=True,
                width_shift_range=0.2,
                height_shift_range=0.2,
                shear_range=0.2,
                zoom_range=0.2,
                **datagenerator_kwargs
            )
        else:
            train_datagenerator = valid_datagenerator

        self.train_generator = train_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="training",
            shuffle=True,
            **dataflow_kwargs
        )

    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)

    def train(self):
        self.steps_per_epoch = self.train_generator.samples // self.train_generator.batch_size
        self.validation_steps = self.valid_generator.samples // self.valid_generator.batch_size

        # ✅ Recompile model with a new optimizer
        self.model.compile(
            optimizer=tf.keras.optimizers.SGD(learning_rate=self.config.params_learning_rate),
            loss=tf.keras.losses.CategoricalCrossentropy(),
            metrics=["accuracy"]
        )

        self.model.fit(
            self.train_generator,
            epochs=self.config.params_epochs,
            steps_per_epoch=self.steps_per_epoch,
            validation_steps=self.validation_steps,
            validation_data=self.valid_generator
        )

        self.save_model(
            path=self.config.trained_model_path,
            model=self.model
        )


In [12]:
import tensorflow as tf
tf.config.run_functions_eagerly(True)


In [13]:
print("Eager execution:", tf.executing_eagerly())


Eager execution: True


In [14]:
from cnnClassifier.config.configuration import ConfigurationManager

config = ConfigurationManager()
print(config.config)     # ✅ should show the content of config.yaml
print(config.params)     # ✅ should show the content of params.yaml


[2025-07-09 00:04:45,004: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-07-09 00:04:45,009: INFO: common: yaml file: params.yaml loaded successfully]
[2025-07-09 00:04:45,012: INFO: common: created directory at: artifacts]
{'artifacts_root': 'artifacts', 'data_ingestion': {'root_dir': 'artifacts/data_ingestion', 'source_URL': 'https://drive.google.com/file/d/1z0mreUtRmR-P-magILsDR3T7M6IkGXtY/view?usp=sharing', 'local_data_file': 'artifacts/data_ingestion/data.zip', 'unzip_dir': 'artifacts/data_ingestion'}, 'prepare_base_model': {'root_dir': 'artifacts/prepare_base_model', 'base_model_path': 'artifacts/prepare_base_model/base_model.h5', 'updated_base_model_path': 'artifacts/prepare_base_model/base_model_updated.h5'}, 'training': {'root_dir': 'artifacts/training', 'trained_model_path': 'artifacts/training/model.h5', 'training_data': 'artifacts/data_ingestion/unzipped_data'}}
{'IMAGE_SIZE': [224, 224, 3], 'BATCH_SIZE': 32, 'EPOCHS': 10, 'CLASSES': 2, 'INCLUDE_TOP'

In [15]:
config = ConfigurationManager()
training_config = config.get_training_config()


[2025-07-09 00:04:46,056: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-07-09 00:04:46,058: INFO: common: yaml file: params.yaml loaded successfully]
[2025-07-09 00:04:46,059: INFO: common: created directory at: artifacts]
[2025-07-09 00:04:46,061: INFO: common: created directory at: artifacts\training]


In [16]:
try:
    config = ConfigurationManager()
    training_config = config.get_training_config()
    training = Training(config=training_config)
    training.get_base_model()
    training.train_valid_generator()
    training.train()
    
except Exception as e:
    raise e

[2025-07-09 00:04:48,615: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-07-09 00:04:48,619: INFO: common: yaml file: params.yaml loaded successfully]
[2025-07-09 00:04:48,621: INFO: common: created directory at: artifacts]
[2025-07-09 00:04:48,623: INFO: common: created directory at: artifacts\training]
[2025-07-09 00:04:48,894: WARNING: saving_utils: Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.]
Found 68 images belonging to 2 classes.
Found 275 images belonging to 2 classes.


c:\Users\Ayaz Khan\Desktop\Chest-Cancer-Classification\.venv\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10


c:\Users\Ayaz Khan\Desktop\Chest-Cancer-Classification\.venv\Lib\site-packages\tensorflow\python\data\ops\structured_function.py:258: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


8/8 ━━━━━━━━━━━━━━━━━━━━ 82s 10s/step - accuracy: 0.5025 - loss: 11.3230 - val_accuracy: 0.3906 - val_loss: 14.6770
Epoch 2/10
1/8 ━━━━━━━━━━━━━━━━━━━━ 1:23 12s/step - accuracy: 0.3750 - loss: 15.4741

c:\Users\Ayaz Khan\Desktop\Chest-Cancer-Classification\.venv\Lib\site-packages\keras\src\trainers\epoch_iterator.py:116: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


8/8 ━━━━━━━━━━━━━━━━━━━━ 18s 895ms/step - accuracy: 0.3750 - loss: 15.4741 - val_accuracy: 0.6094 - val_loss: 22.5610
Epoch 3/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 97s 12s/step - accuracy: 0.4993 - loss: 20.3203 - val_accuracy: 0.7344 - val_loss: 0.6137
Epoch 4/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 17s 861ms/step - accuracy: 0.7188 - loss: 1.4666 - val_accuracy: 0.6094 - val_loss: 11.0506
Epoch 5/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 99s 12s/step - accuracy: 0.5523 - loss: 12.0808 - val_accuracy: 0.3906 - val_loss: 13.8035
Epoch 6/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 14s 946ms/step - accuracy: 0.4211 - loss: 13.5965 - val_accuracy: 0.6094 - val_loss: 14.3699
Epoch 7/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 99s 12s/step - accuracy: 0.6780 - loss: 8.8730 - val_accuracy: 0.8281 - val_loss: 0.4345
Epoch 8/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 18s 885ms/step - accuracy: 0.7188 - loss: 1.8113 - val_accuracy: 0.6094 - val_loss: 4.8990
Epoch 9/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 97s 12s/step - accuracy: 0.6339 - loss: 6.9517 - val_accuracy: 0.8750 - val_loss: 0.